# Лабораторная работа 7. Управление ресурсами в однопроцессорной системе с неоднородными заявками
Выполнил: Пакало Александр Сергеевич, студент РТ5-81Б

## Задание 1
В однопроцессорную систему случайным образом поступают на выполнение $m$
разных типов программ, отличающихся известной трудоемкостью
$Q_1, Q_2,\dots, Q_m$.
Входящий поток простейший с интенсивностью $\lambda$.

Представив данную систему как одноканальную СМО с неограниченной очередью,
вычислить среднее время обслуживания программ, считая длительность
обслуживания случайной величиной (теоретически и экспериментально).

Реализовать алгоритм SPT, выбирая из очереди заявки в соответствии с их
приоритетом по трудоемкости. Рассчитать среднее время обслуживания программ.
Сравнить полученные результаты.

Реализовать алгоритм RR при заданном кванте времени $q$. Оценить среднее время
обслуживания программ. Сравнить полученные результаты. Выяснить, как влияет
величина кванта на среднее время обслуживания программ.

In [226]:
Variant <- 5
set.seed(Variant)
m <- sample(c(6:20), 1)
lambda <- runif(1, 0.1, 2)
Q <- rexp(m, 0.3)
q <- sample(c(1:4), 1)
View(data.frame(m, q, lambda))
print(Q)

m,q,lambda
<int>,<int>,<dbl>
7,1,1.401915


[1] 0.2417931 1.3403831 0.1863999 2.0529013 0.2552007 2.9172096 6.6709868


Заведем таблицу результатов

In [229]:
results <- data.frame(0, 0, 0, 0)
colnames(results) <- c("M/M/1/infty theoretical", "M/M/1/infty practical", "SPT", "Round Robin")
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>
0,0,0,0


### СМО вида $М/М/1/\infty$
Представим данную систему как одноканальную СМО с неограниченной очередью.

#### Теоретически

In [230]:
t2 <- mean(Q)
mu <- 1 / t2
mu

[1] 0.5122623

In [231]:
y <- lambda / mu
y

[1] 2.736714

Так как $y > 1$, поменяем $\lambda$.

In [232]:
lambda <- 0.3

In [233]:
t2 <- mean(Q)
mu <- 1 / t2
mu

[1] 0.5122623

In [234]:
y <- lambda / mu
y

[1] 0.5856375

In [235]:
results[1] <- 1 / mu / (1 - y)
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>
4.711152,0,0,0


#### Численно

In [236]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

env <- simmer("SuperDuperSim")
env

simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }

In [237]:
programs <- trajectory("programs' path") %>%
    seize("server", amount = 1) %>%
    timeout(function() rexp(1, mu)) %>%
    release("server", amount = 1)

In [238]:
SIMULATION_TIME <- 10000

env %>%
    add_resource("server", 1) %>%
    add_generator("programs", programs, function() rexp(1, lambda)) %>%
    run(until = SIMULATION_TIME)

simmer environment: SuperDuperSim | now: 10000 | next: 10003.7234989748
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: programs | monitored: 1 | n_generated: 3029 }

In [239]:
arrivals <- env %>%
    get_mon_arrivals()
arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programs0,1.282143,4.177776,2.89563311,TRUE,1
programs1,1.770443,4.258100,0.08032395,TRUE,1
programs2,4.408824,11.193970,6.78514566,TRUE,1
programs3,7.054637,13.693605,2.49963501,TRUE,1
programs4,11.334202,14.437538,0.74393268,TRUE,1
programs5,13.441375,18.684576,4.24703859,TRUE,1
programs6,19.294948,19.813446,0.51849776,TRUE,1
programs7,21.252343,21.511088,0.25874508,TRUE,1
programs8,25.357193,27.734487,2.37729349,TRUE,1


In [240]:
results[2] <- mean(arrivals %>% with(end_time - start_time))
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>
4.711152,4.906583,0,0


### Алгоритм SPT

In [246]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

spt.env <- simmer("SuperDuperSptSim")
spt.env

simmer environment: SuperDuperSptSim | now: 0 | next: 
{ Monitor: in memory }

Добавим $m$ генераторов. Каждый будет иметь приоритет в зависимости от
скорости выполнения программы. Генератор, создающий программы с наибольшей
длительностью выполнения, будет иметь наименьший приоритет.

In [253]:
SIMULATION_TIME <- 100000

spt.env %>%
    add_resource("server", 1)

Q_sorted_decr <- sort(Q, decreasing = TRUE)

spt.programs_trajectory <- function(time_to_execute) {
    return(
        trajectory("programs' path") %>%
            seize("server", 1) %>%
            timeout(function() rexp(1, 1 / time_to_execute)) %>%
            release("server", 1)
    )
}

for (Q_i in seq_along(Q_sorted_decr)) {
    priority <- Q_i

    name <- paste0("programs", Q_i)

    spt.env %>% add_generator(
        name,
        spt.programs_trajectory(Q_sorted_decr[Q_i]),
        priority = priority,
        preemptible = priority,
        distribution = function() rexp(1, lambda / m)
    )
}

spt.env %>%
    run(until = SIMULATION_TIME)

Warning message in add_resource_(.env$sim_obj, i, positive(capacity), positive(queue_size), :
“resource 'server' already defined”


simmer environment: SuperDuperSptSim | now: 10000 | next: 10000.7444924952
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: programs1 | monitored: 1 | n_generated: 419 }
{ Source: programs2 | monitored: 1 | n_generated: 420 }
{ Source: programs3 | monitored: 1 | n_generated: 417 }
{ Source: programs4 | monitored: 1 | n_generated: 452 }
{ Source: programs5 | monitored: 1 | n_generated: 425 }
{ Source: programs6 | monitored: 1 | n_generated: 404 }
{ Source: programs7 | monitored: 1 | n_generated: 406 }

Warning message in add_generator_(.env$sim_obj, i, trajectory, distribution, mon, :
“process 'programs1' already defined”
Warning message in add_generator_(.env$sim_obj, i, trajectory, distribution, mon, :
“process 'programs2' already defined”
Warning message in add_generator_(.env$sim_obj, i, trajectory, distribution, mon, :
“process 'programs3' already defined”
Warning message in add_generator_(.env$sim_obj, i, trajectory, distribution, mon, :
“process 'programs4' already defined”
Warning message in add_generator_(.env$sim_obj, i, trajectory, distribution, mon, :
“process 'programs5' already defined”
Warning message in add_generator_(.env$sim_obj, i, trajectory, distribution, mon, :
“process 'programs6' already defined”
Warning message in add_generator_(.env$sim_obj, i, trajectory, distribution, mon, :
“process 'programs7' already defined”


simmer environment: SuperDuperSptSim | now: 1e+05 | next: 100000.396731431
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: programs1 | monitored: 1 | n_generated: 4278 }
{ Source: programs2 | monitored: 1 | n_generated: 4212 }
{ Source: programs3 | monitored: 1 | n_generated: 4228 }
{ Source: programs4 | monitored: 1 | n_generated: 4283 }
{ Source: programs5 | monitored: 1 | n_generated: 4328 }
{ Source: programs6 | monitored: 1 | n_generated: 4217 }
{ Source: programs7 | monitored: 1 | n_generated: 4198 }

In [251]:
spt.arrivals <- spt.env %>%
    get_mon_arrivals()
spt.arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programs20,5.369646,5.456885,0.087238432,TRUE,1
programs40,11.208208,11.245917,0.037709016,TRUE,1
programs21,13.728700,14.131183,0.402482958,TRUE,1
programs50,14.359118,14.505468,0.146350310,TRUE,1
programs30,28.487273,28.787825,0.300551426,TRUE,1
programs22,31.953079,31.971580,0.018501405,TRUE,1
programs60,32.701031,32.774976,0.073944568,TRUE,1
programs51,33.627934,33.791295,0.163361372,TRUE,1
programs70,36.289504,36.372560,0.083056266,TRUE,1


In [252]:
results[3] <- mean(spt.arrivals %>% with(end_time - start_time))
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>
4.711152,4.906583,0.1928865,0


### Алгоритм Round Robin
Реализуем Round Robin с помощью simmer, воспользовавшись механизмом select,
выбирающим из очереди значение по определенной стратегии.

In [170]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

env <- simmer("SuperDuperSim")
env

simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }

In [171]:
programs <- trajectory("programs' path") %>%
    select("server", "round-robin") %>%
    seize_selected(1) %>%
    timeout(function() rexp(1, mu)) %>%
    release_selected(1)

In [172]:
SIMULATION_TIME <- 10000

env %>%
    add_resource("server", 1) %>%
    add_generator("programs", programs, function() rexp(1, lambda)) %>%
    run(until = SIMULATION_TIME)

simmer environment: SuperDuperSim | now: 10000 | next: 10000.6880190946
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: programs | monitored: 1 | n_generated: 2958 }

In [173]:
arrivals <- env %>%
    get_mon_arrivals()
arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programs0,11.71948,11.72665,0.007171833,TRUE,1
programs1,11.97214,16.47420,4.502058210,TRUE,1
programs2,13.18187,19.27042,2.796220815,TRUE,1
programs3,13.84978,19.41637,0.145955088,TRUE,1
programs4,17.25988,19.73762,0.321247764,TRUE,1
programs5,17.36269,21.07808,1.340459151,TRUE,1
programs6,18.08577,21.35824,0.280164119,TRUE,1
programs7,32.47119,33.76257,1.291385800,TRUE,1
programs8,34.91324,35.59000,0.676756013,TRUE,1


In [185]:
results[5] <- mean(arrivals %>% with(end_time - start_time))
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT theoretical,SPT practical,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4.711152,4.545182,4.008172,2.5598,4.91254


Как видно, практические вычисления совпадают теоретическими с некоторой
погрешностью, которая уменьшается при увеличении числа экспериментов.

При этом система, выполненная с помощью алгоритма Round Robin оказалась
быстрее обычной системы, а система, реализованная с алгоритмом SPT - самой
быстрой.